In [1]:
import requests # make http requests
import json # make sense of what the requests return
import pickle # save our data to our computer

import pandas as pd # structure out data
import numpy as np # do a bit of math
import scipy.stats as sci # do a bit more math

from datetime import datetime # make working with dates 1000x easier 
import time # become time lords
import os

import threading #responsible for the paraleliation of the code

import warnings
import sqlite3
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
cookie = {'steamLoginSecure': '76561198076577233%7C%7CB2032A4F108686E696F49EA2565CD17ADAE70109'}

In [24]:
ids = []
with open('getIds/ids.txt') as f:
    for line in f.readlines():
        ids.append(line.strip())
gameList = ids

In [4]:
global requestCnt
requestCnt = 0
global requestsBefore429
requestsBefore429 = 38
global log
log = ''
global mutex
mutex = threading.Lock() 


In [5]:
def createDb():
    conn = sqlite3.connect('test_database') 
#   
#     c.execute('''
#               CREATE TABLE IF NOT EXISTS products
#               ([product_id] INTEGER PRIMARY KEY, [product_name] TEXT)
#               ''')

#     c.execute('''
#               CREATE TABLE IF NOT EXISTS prices
#               ([product_id] INTEGER PRIMARY KEY, [price] INTEGER)
#               ''')

    conn.commit()
    return conn

In [6]:
global conn
conn = createDb()

In [7]:
def on429Error(url):
    global requestsBefore429
#     global mutex
    print('##### Got 429 Error, sleeping for 1 minute #####')
#     with mutex:
    #requestsBefore429 -= 1
    print(f'##### Decreasing requestsBefore429 to ${requestsBefore429}#####')
    time.sleep(60*5)
    response = requests.get(url, cookies=cookie); # get page
    if response.status_code == 429:
        on429Error(url)
    else:
        return response

In [8]:
def checkRequestCntAndSleep():
    time.sleep(2.5)
    global requestCnt
    global requestsBefore429
#     global mutex
#     with mutex:
    requestCnt +=1
    if requestCnt == requestsBefore429:
        requestCnt = 0
        time.sleep(60)

In [38]:
def getAllItemsNames(ids):
    for gameID in ids:
        global log
        #checkRequestCntAndSleep()
        time.sleep(2.5)
        allItemNames = []
        # find total number items
        url = 'https://steamcommunity.com/market/search/render/?search_descriptions=0&sort_column=default&sort_dir=desc&appid='+gameID+'&norender=1&count=100'
        allItemsGet = requests.get(url, cookies=cookie); # get page
        if allItemsGet.status_code == 429:
            allItemsGet = on429Error(url)
        if not allItemsGet:
            log += f'!allItems on gameID={gameID}, content null \n'
            continue
        try:
            allItems = allItemsGet.json()
            results = allItems['results']
            totalItems = allItems['total_count']
        except:
            print(f'!Erro no try catch do getAllItemNames gameid={gameID} \n')
            continue

        partial = 0

        #     you can only get 100 items at a time (despite putting in count= >100)
        #     so we have to loop through in batches of 100 to get every single item name by specifying the start position
        for currPos in range(0,totalItems+90,90): # loop through all items
#             checkRequestCntAndSleep()
            time.sleep(2.5)
            # get item name of each
            results = []
            url ='https://steamcommunity.com/market/search/render/?start='+str(currPos)+'&count=100&search_descriptions=0&sort_column=default&sort_dir=desc&appid='+gameID+'&norender=1&count=5000'
            allItemsGet = requests.get(url, cookies=cookie);
            if allItemsGet.status_code == 429:
                allItemsGet = on429Error(url)
            print('Items '+str(currPos)+' out of '+str(totalItems)+' code: '+str(allItemsGet.status_code)+ ' on Game:'+ gameID) # reassure us the code is running and we are getting good returns (code 200)

            if not allItemsGet:
                log += f'!allItems on gameID={gameID}, content null \n'
                continue
            try:
                allItems = allItemsGet.json()
                results = allItems['results']
            except:
                print(f'!Erro no try catch do getAllItemNames gameid={gameID}, pos={currPos} \n')

            for currItem in results: 
                allItemNames.append(currItem['hash_name']) # save the names
                with open('allItemsNamestxt/'+gameID+'.txt', 'a') as file:
                    file.write(currItem['hash_name'] + '\n')
            if currPos != 0 and currPos % 50000 == 0:
                allItemNames = list(set(allItemNames))
                with open('allItemsNames/ItemNames_'+gameID+'_partial_'+str(partial)+'.pkl', "wb") as file: # change the text file name to whatever you want
                    pickle.dump(allItemNames, file)
                partial +=1
                allItemNames = []

        # remove dupes by converting from list to set and back again
        print('len(list(set(allItemNames))): ', len(list(set(allItemNames))),'len(allItemNames): ', len(allItemNames))
        allItemNames = list(set(allItemNames))

        # Save all the name so we don't have to do this step anymore
        # use pickle to save all the names so i dont have to keep running above code
        #         if(os.path.isdir('./allItemsNames') == False):
        #             print('n achou o diretorio')
        #             os.mkdir('allItemsNames')

        with open('allItemsNames/ItemNames_'+gameID+'.pkl', "wb") as file: # change the text file name to whatever you want
            pickle.dump(allItemNames, file)

In [11]:
# open file with all names
def creating_dataframes(today = datetime.today().strftime("%m/%d/%y")):

    date_rng = pd.date_range(start='07/01/2012', end=today, freq='D') # because tf2 key
    marketIndexMaster_price = pd.DataFrame(date_rng, columns=['date'])
    marketIndexMaster_price = marketIndexMaster_price.set_index('date')

    marketIndexMaster_delta = pd.DataFrame(date_rng, columns=['date'])
    marketIndexMaster_delta = marketIndexMaster_delta.set_index('date')

    marketIndexMaster_vol = pd.DataFrame(date_rng, columns=['date'])
    marketIndexMaster_vol = marketIndexMaster_vol.set_index('date')

    # intialize our Panda's dataframe with the data we want from each item
    allItemsPD = pd.DataFrame(data=None,index=None,columns = ['itemName','initial','timeOnMarket','priceIncrease','smoothChange','priceAvg','priceSD','maxPrice','maxIdx','minPrice','minIdx','swing','volAvg','volSD','slope','rr'])
    
    return marketIndexMaster_price, marketIndexMaster_delta, marketIndexMaster_vol, allItemsPD

In [12]:
# need to encode symbols into ASCII for http (https://www.w3schools.com/tags/ref_urlencode.asp)
def ascii_to_http(currItem):
    currItemHTTP = currItem.replace(' ','%20') # convert spaces to %20
    currItemHTTP = currItemHTTP.replace('&','%26')  # convert & to %26
    currItemHTTP = currItemHTTP.replace("'",'%27')  # convert ' to %27
    currItemHTTP = currItemHTTP.replace("(",'%28')  # convert ' to %27
    currItemHTTP = currItemHTTP.replace(")",'%29')  # convert ' to %27
    currItemHTTP = currItemHTTP.replace("|",'%7C')  # convert ' to %27
    currItemHTTP = currItemHTTP.replace(",",'%2C')  # convert ' to %27
    
    return currItemHTTP

In [43]:
#paralelized request of items price history and the respective calculations needed
def requests_handler(alocatedItemNames, gameID):
    global log
    global marketIndex
    global conn
    currRun = 1
    for currItem in alocatedItemNames: # go through all item names
        #checkRequestCntAndSleep()
        time.sleep(2.5)
        currRun +=1
            
        currItemHTTP = ascii_to_http(currItem)

        # Just to get a simpler request
        itemUrl = 'https://steamcommunity.com/market/pricehistory/?appid='+gameID+'&market_hash_name='+currItemHTTP

        #whitout the try/except the program was getting interrupted when trying to get csgo items
        try:
            item = requests.get(itemUrl, cookies=cookie)  # get item data
        except requests.exceptions.ConnectionError as e:
            item = "No response"
            
        if item == "No response":
            print(f'{currItemHTTP} bad no data \n')
            continue
        if item.status_code == 429:
                item = on429Error(itemUrl)
        
        print(str(currRun),' out of ',str(len(alocatedItemNames))+' code: '+str(item.status_code) + ' on Game:'+ gameID + ' item: ' + currItem) 

        if item.status_code == 200:
            item = item.content 
            item = json.loads(item) 
            if item: # did we even get any data back
                itemPriceData = item['prices'] # is there price data?
                if itemPriceData == False or not itemPriceData: # if there was an issue with the request then data will return false and the for loop will just continue to the next item
                    print(f'{currItemHTTP} bad no data \n')
                    continue               # this could be cause the http item name was weird (eg symbol not converted to ASCII) but it will also occur if you make too many requests too fast (this is handled below)
                else:
                    # initialize stuff
                    itemPrices = []  # steam returns MEDIAN price for given time bin
                    itemVol = [] 
                    itemDate = [] 
                    for currDay in itemPriceData: # pull out the actual data
                        itemPrices.append(currDay[1])  # idx 1 is price
                        itemVol.append(currDay[2])  # idx 2 is volume of items sold
                        itemDate.append(datetime.strptime(currDay[0][0:11], '%b %d %Y')) # idx 0 is the date

                    # lists are strings, convert to numbers
                    itemPrices = list(map(float, itemPrices)) 
                    itemVol = list(map(int, itemVol)) 

                    # combine sales that occurs on the same day
                    # avg prices, sum volume
                    # certainly not the best way to do this but, whatever
                    for currDay in range(len(itemDate)-1,0,-1): # start from end (-1) and go to start
                        if itemDate[currDay] == itemDate[currDay-1]: # if current element's date same as the one before it
                            itemPrices[currDay-1] = np.median([itemPrices[currDay],itemPrices[currDay-1]])  # average prices from the two days
                            itemVol[currDay-1] = np.sum([itemVol[currDay],itemVol[currDay-1]])  # sum volume
                            # delete the repeats
                            del itemDate[currDay] 
                            del itemVol[currDay] 
                            del itemPrices[currDay]

                    # now that days are combined
                    normTime = list(range(0,len(itemPrices)))  # create a new list that "normalizes" days from 0 to n, easier to work with than datetime

                    # some basic data
                    timeOnMarket = (datetime.today()-itemDate[0]).days  # have to do this because if sales are spare day[0] could be months/years ago
                    priceIncrease = itemPrices[-1] -itemPrices[0]  # what was the price increase from day 0 to the most recent day [-1]
                    maxPrice = max(itemPrices)  # max price
                    maxIdx = itemPrices.index(maxPrice)  # when was the max price?
                    minPrice = min(itemPrices) 
                    minIdx = itemPrices.index(minPrice) 
                    swing = maxPrice - minPrice  # greatest price swing

                    if timeOnMarket >= 30:
                        smoothStart = np.mean(itemPrices[0:10]) 
                        smoothEnd = np.mean(itemPrices[-11:-1])
                        smoothChange = smoothEnd - smoothStart 
                    else:
                        smoothStart = np.nan 
                        smoothEnd = np.nan 
                        smoothChange = np.nan 


                    # get some descriptive stats
                    itemPriceAvg = np.mean(itemPrices)  # average price
                    if len(itemPrices) > 1: # make sure there is at least two days of sales
                        itemPriceInitial = itemPrices[1] - itemPrices[0]  # how much did the price jump from day 0 to 1? eg the first trading day
                    else:
                        itemPriceInitial = itemPrices[0] 
                    itemVolAvg = np.mean(itemVol) 

                    itemPriceSD = np.std(itemPrices) 
                    itemVolSD = np.std(itemVol) 


                    # linear regression to find slope and fit
                    fitR = sci.linregress(normTime,itemPrices)  # slope intercept rvalue pvalue stderr
                    RR = float(fitR[2]**2)  # convert to R^2 value


                    # stock market

                    stock = pd.DataFrame(itemDate,columns=['date']) 
                    stock = stock.set_index('date') 
                    stock['price'] = itemPrices 
                    stock['vol'] = itemVol 

                    stock['delta'] = stock['price'].diff() 
#                     with mutex:
                    marketIndex.append((currItem, [stock['price'] , stock['vol'] ,stock['delta'] ]))

                    # save data 
                    currentItemDict = {'itemName':currItem,'initial':itemPriceInitial,'timeOnMarket':timeOnMarket,'priceIncrease':priceIncrease,'smoothChange':smoothChange,'priceAvg':itemPriceAvg,'priceSD':itemPriceSD,'maxPrice':maxPrice,'maxIdx':maxIdx,'minPrice':minPrice,'minIdx':minIdx,'swing':swing,'volAvg':itemVolAvg,'volSD':itemVolSD,'slope':fitR[0],'rr':RR}
#                     with mutex:
                    aditions.append(currentItemDict)
        else:
            print(f'{currItemHTTP} bad no data \n')
            continue
        

def threads_exec(gameID):
#     global mutex             
                
    with open('allItemsNames/ItemNames_'+gameID+'.pkl', "rb") as file:   # Unpickling
        allItemNames = pickle.load(file)

    #pandas dataframes are not thread safe, so we need to save data in other structures that are        
    global aditions
    aditions = []
    global marketIndex 
    marketIndex = []

#     threadsArray = []
#     num_threads = 100 
#     works = [list(allItemNames)[i::num_threads] for i in range(num_threads)] # #spread the work among threads
    
#     i = 0
#     for alocatedItemNames in works: #make each thread deal with it's share of  data
#         threadsArray.append(threading.Thread(target=requests_handler, args=(alocatedItemNames,))) # iniciar cada uma
#         threadsArray[i].start()
#         i+=1
        
#     i = 0
#     for _ in works: 
#         threadsArray[i].join()
#         i+=1
    
    requests_handler(allItemNames, gameID)
    print("done with the scrapping moving to pass data to dataframes")

    return aditions, marketIndex

def complete_game_processing(gameID, end = None):
    global conn
    if end:
        marketIndexMaster_price, marketIndexMaster_delta, marketIndexMaster_vol, allItemsPD = creating_dataframes(end)
    else:
        #default argument is today's date
        marketIndexMaster_price, marketIndexMaster_delta, marketIndexMaster_vol, allItemsPD = creating_dataframes()
    
    aditions, marketIndex = threads_exec(gameID)

    for d in aditions:
        currItemPD = pd.DataFrame(d,index=[0]) 
        allItemsPD= allItemsPD.append(currItemPD,ignore_index=True) 

    for pair in marketIndex:
        marketIndexMaster_price[pair[0]] = pair[1][0]
        marketIndexMaster_vol[pair[0]] = pair[1][1]
        marketIndexMaster_delta[pair[0]] = pair[1][2]

    print("proceding to write data in files")
    # save the dataframe
    os.mkdir(gameID)
    #allItemsPD_.to_sql(f'allItemsPD_{gameID}', con=conn)
    allItemsPD.to_pickle(gameID+'/'+gameID+'PriceData.pkl') 
    marketIndexMaster_price.to_pickle(gameID+'/'+gameID+'marketPrice.pkl') 
    #marketIndexMaster_price.to_sql(f'marketIndexMaster_price_{gameID}', con=conn)
    marketIndexMaster_vol.to_pickle(gameID+'/'+gameID+'marketVol.pkl') 
    #marketIndexMaster_vol.to_sql(f'marketIndexMaster_vol_{gameID}', con=conn)
    marketIndexMaster_delta.to_pickle(gameID+'/'+gameID+'marketDelta.pkl') 
    #marketIndexMaster_delta.to_sql(f'marketIndexMaster_delta_{gameID}', con=conn)
    
    conn.commit()
    print('Saved '+gameID)

In [14]:
#getAllItemsNames(ids)

In [15]:
#ids2 = ids[ids.index('321360'):]

In [16]:
#getAllItemsNames(ids2)

In [20]:
ids3 = ids[ids.index('753')+1:]
getAllItemsNames(ids3)

Items 0 out of 161 code: 200 on Game:250820
Items 20 out of 161 code: 200 on Game:250820
Items 40 out of 161 code: 200 on Game:250820
Items 60 out of 161 code: 200 on Game:250820
Items 80 out of 161 code: 200 on Game:250820
Items 100 out of 161 code: 200 on Game:250820
Items 120 out of 161 code: 200 on Game:250820
Items 140 out of 161 code: 200 on Game:250820
Items 160 out of 161 code: 200 on Game:250820
Items 180 out of 161 code: 200 on Game:250820
Items 0 out of 15 code: 200 on Game:1403280
Items 20 out of 15 code: 200 on Game:1403280
Items 0 out of 8 code: 200 on Game:264710
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 20 out of 8 code: 200 on Game:264710
Items 0 out of 13 code: 200 on Game:418030
Items 20 out of 13 code: 200 on Game:418030
Items 0 out of 301 code: 200 on Game:321400
Items 20 out of 301 code: 200 on Game:321400
Items 40 out of 301 code: 200 on Game:321400
Items 60 out of 301 code: 200 on Game:321400
Items 80 o

Items 2500 out of 40871 code: 200 on Game:440
Items 2520 out of 40871 code: 200 on Game:440
Items 2540 out of 40871 code: 200 on Game:440
Items 2560 out of 40871 code: 200 on Game:440
Items 2580 out of 40871 code: 200 on Game:440
Items 2600 out of 40871 code: 200 on Game:440
Items 2620 out of 40871 code: 200 on Game:440
Items 2640 out of 40871 code: 200 on Game:440
Items 2660 out of 40871 code: 200 on Game:440
Items 2680 out of 40871 code: 200 on Game:440
Items 2700 out of 40871 code: 200 on Game:440
Items 2720 out of 40871 code: 200 on Game:440
Items 2740 out of 40871 code: 200 on Game:440
Items 2760 out of 40871 code: 200 on Game:440
Items 2780 out of 40871 code: 200 on Game:440
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 2800 out of 40871 code: 200 on Game:440
Items 2820 out of 40871 code: 200 on Game:440
Items 2840 out of 40871 code: 200 on Game:440
Items 2860 out of 40871 code: 200 on Game:440
Items 2880 out of 40871 code: 

Items 5800 out of 40871 code: 200 on Game:440
Items 5820 out of 40871 code: 200 on Game:440
Items 5840 out of 40871 code: 200 on Game:440
Items 5860 out of 40871 code: 200 on Game:440
Items 5880 out of 40871 code: 200 on Game:440
Items 5900 out of 40871 code: 200 on Game:440
Items 5920 out of 40871 code: 200 on Game:440
Items 5940 out of 40871 code: 200 on Game:440
Items 5960 out of 40871 code: 200 on Game:440
Items 5980 out of 40871 code: 200 on Game:440
Items 6000 out of 40871 code: 200 on Game:440
Items 6020 out of 40871 code: 200 on Game:440
Items 6040 out of 40871 code: 200 on Game:440
Items 6060 out of 40871 code: 200 on Game:440
Items 6080 out of 40871 code: 200 on Game:440
Items 6100 out of 40871 code: 200 on Game:440
Items 6120 out of 40871 code: 200 on Game:440
Items 6140 out of 40871 code: 200 on Game:440
Items 6160 out of 40871 code: 200 on Game:440
Items 6180 out of 40871 code: 200 on Game:440
Items 6200 out of 40871 code: 200 on Game:440
Items 6220 out of 40871 code: 200 

Items 9120 out of 40871 code: 200 on Game:440
Items 9140 out of 40871 code: 200 on Game:440
Items 9160 out of 40871 code: 200 on Game:440
Items 9180 out of 40871 code: 200 on Game:440
Items 9200 out of 40871 code: 200 on Game:440
Items 9220 out of 40871 code: 200 on Game:440
Items 9240 out of 40871 code: 200 on Game:440
Items 9260 out of 40871 code: 200 on Game:440
Items 9280 out of 40871 code: 200 on Game:440
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 9300 out of 40871 code: 200 on Game:440
Items 9320 out of 40871 code: 200 on Game:440
Items 9340 out of 40871 code: 200 on Game:440
Items 9360 out of 40871 code: 200 on Game:440
Items 9380 out of 40871 code: 200 on Game:440
Items 9400 out of 40871 code: 200 on Game:440
Items 9420 out of 40871 code: 200 on Game:440
Items 9440 out of 40871 code: 200 on Game:440
Items 9460 out of 40871 code: 200 on Game:440
Items 9480 out of 40871 code: 200 on Game:440
Items 9500 out of 40871 code: 

Items 12340 out of 40871 code: 200 on Game:440
Items 12360 out of 40871 code: 200 on Game:440
Items 12380 out of 40871 code: 200 on Game:440
Items 12400 out of 40871 code: 200 on Game:440
Items 12420 out of 40871 code: 200 on Game:440
Items 12440 out of 40871 code: 200 on Game:440
Items 12460 out of 40871 code: 200 on Game:440
Items 12480 out of 40871 code: 200 on Game:440
Items 12500 out of 40871 code: 200 on Game:440
Items 12520 out of 40871 code: 200 on Game:440
Items 12540 out of 40871 code: 200 on Game:440
Items 12560 out of 40871 code: 200 on Game:440
Items 12580 out of 40871 code: 200 on Game:440
Items 12600 out of 40871 code: 200 on Game:440
Items 12620 out of 40871 code: 200 on Game:440
Items 12640 out of 40871 code: 200 on Game:440
Items 12660 out of 40871 code: 200 on Game:440
Items 12680 out of 40871 code: 200 on Game:440
Items 12700 out of 40871 code: 200 on Game:440
Items 12720 out of 40871 code: 200 on Game:440
Items 12740 out of 40871 code: 200 on Game:440
Items 12760 o

...

Items 2680 out of 12871 code: 200 on Game:304930
Items 2700 out of 12871 code: 200 on Game:304930
Items 2720 out of 12871 code: 200 on Game:304930
Items 2740 out of 12871 code: 200 on Game:304930
Items 2760 out of 12871 code: 200 on Game:304930
Items 2780 out of 12871 code: 200 on Game:304930
Items 2800 out of 12871 code: 200 on Game:304930
Items 2820 out of 12871 code: 200 on Game:304930
Items 2840 out of 12871 code: 200 on Game:304930
Items 2860 out of 12871 code: 200 on Game:304930
Items 2880 out of 12871 code: 200 on Game:304930
Items 2900 out of 12871 code: 200 on Game:304930
Items 2920 out of 12871 code: 200 on Game:304930
Items 2940 out of 12871 code: 200 on Game:304930
Items 2960 out of 12871 code: 200 on Game:304930
Items 2980 out of 12871 code: 200 on Game:304930
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 3000 out of 12871 code: 200 on Game:304930
Items 3020 out of 12871 code: 200 on Game:304930
Items 3040 out of 1287

Items 5800 out of 12871 code: 200 on Game:304930
Items 5820 out of 12871 code: 200 on Game:304930
Items 5840 out of 12871 code: 200 on Game:304930
Items 5860 out of 12871 code: 200 on Game:304930
Items 5880 out of 12871 code: 200 on Game:304930
Items 5900 out of 12871 code: 200 on Game:304930
Items 5920 out of 12871 code: 200 on Game:304930
Items 5940 out of 12871 code: 200 on Game:304930
Items 5960 out of 12871 code: 200 on Game:304930
Items 5980 out of 12871 code: 200 on Game:304930
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 6000 out of 12871 code: 200 on Game:304930
Items 6020 out of 12871 code: 200 on Game:304930
Items 6040 out of 12871 code: 200 on Game:304930
Items 6060 out of 12871 code: 200 on Game:304930
Items 6080 out of 12871 code: 200 on Game:304930
Items 6100 out of 12871 code: 200 on Game:304930
Items 6120 out of 12871 code: 200 on Game:304930
Items 6140 out of 12871 code: 200 on Game:304930
Items 6160 out of 1287

Items 8920 out of 12871 code: 200 on Game:304930
Items 8940 out of 12871 code: 200 on Game:304930
Items 8960 out of 12871 code: 200 on Game:304930
Items 8980 out of 12871 code: 200 on Game:304930
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 9000 out of 12871 code: 200 on Game:304930
Items 9020 out of 12871 code: 200 on Game:304930
Items 9040 out of 12871 code: 200 on Game:304930
Items 9060 out of 12871 code: 200 on Game:304930
Items 9080 out of 12871 code: 200 on Game:304930
Items 9100 out of 12871 code: 200 on Game:304930
Items 9120 out of 12871 code: 200 on Game:304930
Items 9140 out of 12871 code: 200 on Game:304930
Items 9160 out of 12871 code: 200 on Game:304930
Items 9180 out of 12871 code: 200 on Game:304930
Items 9200 out of 12871 code: 200 on Game:304930
Items 9220 out of 12871 code: 200 on Game:304930
Items 9240 out of 12871 code: 200 on Game:304930
Items 9260 out of 12871 code: 200 on Game:304930
Items 9280 out of 1287

##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 12000 out of 12871 code: 200 on Game:304930
Items 12020 out of 12871 code: 200 on Game:304930
Items 12040 out of 12871 code: 200 on Game:304930
Items 12060 out of 12871 code: 200 on Game:304930
Items 12080 out of 12871 code: 200 on Game:304930
Items 12100 out of 12871 code: 200 on Game:304930
Items 12120 out of 12871 code: 200 on Game:304930
Items 12140 out of 12871 code: 200 on Game:304930
Items 12160 out of 12871 code: 200 on Game:304930
Items 12180 out of 12871 code: 200 on Game:304930
Items 12200 out of 12871 code: 200 on Game:304930
Items 12220 out of 12871 code: 200 on Game:304930
Items 12240 out of 12871 code: 200 on Game:304930
Items 12260 out of 12871 code: 200 on Game:304930
Items 12280 out of 12871 code: 200 on Game:304930
Items 12300 out of 12871 code: 200 on Game:304930
Items 12320 out of 12871 code: 200 on Game:304930
Items 12340 out of 12871 code: 200 on Game:304930
Item

Items 160 out of 552 code: 200 on Game:625340
Items 180 out of 552 code: 200 on Game:625340
Items 200 out of 552 code: 200 on Game:625340
Items 220 out of 552 code: 200 on Game:625340
Items 240 out of 552 code: 200 on Game:625340
Items 260 out of 552 code: 200 on Game:625340
Items 280 out of 552 code: 200 on Game:625340
Items 300 out of 552 code: 200 on Game:625340
Items 320 out of 552 code: 200 on Game:625340
Items 340 out of 552 code: 200 on Game:625340
Items 360 out of 552 code: 200 on Game:625340
Items 380 out of 552 code: 200 on Game:625340
Items 400 out of 552 code: 200 on Game:625340
Items 420 out of 552 code: 200 on Game:625340
Items 440 out of 552 code: 200 on Game:625340
Items 460 out of 552 code: 502 on Game:625340
Items 480 out of 552 code: 200 on Game:625340
Items 500 out of 552 code: 200 on Game:625340
Items 520 out of 552 code: 200 on Game:625340
Items 540 out of 552 code: 200 on Game:625340
Items 560 out of 552 code: 200 on Game:625340
Items 0 out of 1 code: 200 on Game

In [22]:
#skipped 753 because its too big
#753 eh o id da steam eem si
getAllItemsNames(['753'])

Items 0 out of 285149 code: 200 on Game:753
Items 20 out of 285149 code: 200 on Game:753
Items 40 out of 285149 code: 200 on Game:753
Items 60 out of 285149 code: 200 on Game:753
Items 80 out of 285149 code: 200 on Game:753
Items 100 out of 285149 code: 200 on Game:753
Items 120 out of 285149 code: 200 on Game:753
Items 140 out of 285149 code: 200 on Game:753
Items 160 out of 285149 code: 200 on Game:753
Items 180 out of 285149 code: 200 on Game:753
Items 200 out of 285149 code: 200 on Game:753
Items 220 out of 285149 code: 200 on Game:753
Items 240 out of 285149 code: 200 on Game:753
Items 260 out of 285149 code: 200 on Game:753
Items 280 out of 285149 code: 200 on Game:753
Items 300 out of 285149 code: 200 on Game:753
Items 320 out of 285149 code: 200 on Game:753
Items 340 out of 285149 code: 200 on Game:753
Items 360 out of 285149 code: 200 on Game:753
Items 380 out of 285149 code: 200 on Game:753
Items 400 out of 285149 code: 200 on Game:753
Items 420 out of 285149 code: 200 on Gam

Items 3280 out of 285149 code: 200 on Game:753
Items 3300 out of 285149 code: 200 on Game:753
Items 3320 out of 285149 code: 200 on Game:753
Items 3340 out of 285149 code: 200 on Game:753
Items 3360 out of 285149 code: 200 on Game:753
Items 3380 out of 285149 code: 200 on Game:753
Items 3400 out of 285149 code: 200 on Game:753
Items 3420 out of 285149 code: 200 on Game:753
Items 3440 out of 285149 code: 200 on Game:753
Items 3460 out of 285149 code: 200 on Game:753
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 3480 out of 285149 code: 200 on Game:753
Items 3500 out of 285149 code: 200 on Game:753
Items 3520 out of 285149 code: 200 on Game:753
Items 3540 out of 285149 code: 200 on Game:753
Items 3560 out of 285149 code: 200 on Game:753
Items 3580 out of 285149 code: 200 on Game:753
Items 3600 out of 285149 code: 200 on Game:753
Items 3620 out of 285149 code: 200 on Game:753
Items 3640 out of 285149 code: 200 on Game:753
Items 3660 

Items 6500 out of 285149 code: 200 on Game:753
Items 6520 out of 285149 code: 200 on Game:753
Items 6540 out of 285149 code: 200 on Game:753
Items 6560 out of 285149 code: 200 on Game:753
Items 6580 out of 285149 code: 200 on Game:753
Items 6600 out of 285149 code: 200 on Game:753
Items 6620 out of 285149 code: 200 on Game:753
Items 6640 out of 285149 code: 200 on Game:753
Items 6660 out of 285149 code: 200 on Game:753
Items 6680 out of 285149 code: 200 on Game:753
Items 6700 out of 285149 code: 200 on Game:753
Items 6720 out of 285149 code: 200 on Game:753
Items 6740 out of 285149 code: 200 on Game:753
Items 6760 out of 285149 code: 200 on Game:753
Items 6780 out of 285149 code: 200 on Game:753
Items 6800 out of 285149 code: 200 on Game:753
Items 6820 out of 285149 code: 200 on Game:753
Items 6840 out of 285149 code: 200 on Game:753
Items 6860 out of 285149 code: 200 on Game:753
Items 6880 out of 285149 code: 200 on Game:753
Items 6900 out of 285149 code: 200 on Game:753
Items 6920 ou

Items 9760 out of 285149 code: 200 on Game:753
Items 9780 out of 285149 code: 200 on Game:753
Items 9800 out of 285149 code: 200 on Game:753
Items 9820 out of 285149 code: 200 on Game:753
Items 9840 out of 285149 code: 200 on Game:753
Items 9860 out of 285149 code: 200 on Game:753
Items 9880 out of 285149 code: 200 on Game:753
Items 9900 out of 285149 code: 200 on Game:753
Items 9920 out of 285149 code: 200 on Game:753
Items 9940 out of 285149 code: 200 on Game:753
Items 9960 out of 285149 code: 200 on Game:753
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 9980 out of 285149 code: 200 on Game:753
Items 10000 out of 285149 code: 200 on Game:753
Items 10020 out of 285149 code: 200 on Game:753
Items 10040 out of 285149 code: 200 on Game:753
Items 10060 out of 285149 code: 200 on Game:753
Items 10080 out of 285149 code: 200 on Game:753
Items 10100 out of 285149 code: 200 on Game:753
Items 10120 out of 285149 code: 200 on Game:753
Item

Items 12940 out of 285149 code: 200 on Game:753
Items 12960 out of 285149 code: 200 on Game:753
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 12980 out of 285149 code: 200 on Game:753
Items 13000 out of 285149 code: 200 on Game:753
Items 13020 out of 285149 code: 200 on Game:753
Items 13040 out of 285149 code: 200 on Game:753
Items 13060 out of 285149 code: 200 on Game:753
Items 13080 out of 285149 code: 200 on Game:753
Items 13100 out of 285149 code: 200 on Game:753
Items 13120 out of 285149 code: 200 on Game:753
Items 13140 out of 285149 code: 200 on Game:753
Items 13160 out of 285149 code: 200 on Game:753
Items 13180 out of 285149 code: 200 on Game:753
Items 13200 out of 285149 code: 200 on Game:753
Items 13220 out of 285149 code: 200 on Game:753
Items 13240 out of 285149 code: 200 on Game:753
Items 13260 out of 285149 code: 200 on Game:753
Items 13280 out of 285149 code: 200 on Game:753
Items 13300 out of 285149 code: 200 on G

Items 16080 out of 285149 code: 200 on Game:753
Items 16100 out of 285149 code: 200 on Game:753
Items 16120 out of 285149 code: 200 on Game:753
Items 16140 out of 285149 code: 200 on Game:753
Items 16160 out of 285149 code: 200 on Game:753
Items 16180 out of 285149 code: 200 on Game:753
Items 16200 out of 285149 code: 200 on Game:753
Items 16220 out of 285149 code: 200 on Game:753
Items 16240 out of 285149 code: 200 on Game:753
Items 16260 out of 285149 code: 200 on Game:753
Items 16280 out of 285149 code: 200 on Game:753
Items 16300 out of 285149 code: 200 on Game:753
Items 16320 out of 285149 code: 200 on Game:753
Items 16340 out of 285149 code: 200 on Game:753
Items 16360 out of 285149 code: 200 on Game:753
Items 16380 out of 285149 code: 200 on Game:753
Items 16400 out of 285149 code: 200 on Game:753
Items 16420 out of 285149 code: 200 on Game:753
Items 16440 out of 285149 code: 200 on Game:753
Items 16460 out of 285149 code: 200 on Game:753
##### Got 429 Error, sleeping for 1 minu

KeyboardInterrupt: 

In [39]:
getAllItemsNames(ids)

Items 0 out of 0 code: 200 on Game:1889410
len(list(set(allItemNames))):  0 len(allItemNames):  0
Items 0 out of 8 code: 200 on Game:777010
Items 90 out of 8 code: 200 on Game:777010
len(list(set(allItemNames))):  8 len(allItemNames):  8
Items 0 out of 354 code: 200 on Game:866510
Items 90 out of 354 code: 200 on Game:866510
Items 180 out of 354 code: 200 on Game:866510
Items 270 out of 354 code: 200 on Game:866510
Items 360 out of 354 code: 200 on Game:866510
len(list(set(allItemNames))):  324 len(allItemNames):  384
Items 0 out of 2204 code: 200 on Game:614910
Items 90 out of 2204 code: 200 on Game:614910
Items 180 out of 2204 code: 200 on Game:614910
Items 270 out of 2204 code: 200 on Game:614910
Items 360 out of 2204 code: 200 on Game:614910
Items 450 out of 2204 code: 200 on Game:614910
Items 540 out of 2204 code: 200 on Game:614910
Items 630 out of 2204 code: 200 on Game:614910
Items 720 out of 2204 code: 200 on Game:614910
Items 810 out of 2204 code: 200 on Game:614910
Items 900

Items 4590 out of 8005 code: 200 on Game:582810
Items 4680 out of 8005 code: 200 on Game:582810
Items 4770 out of 8005 code: 200 on Game:582810
Items 4860 out of 8005 code: 200 on Game:582810
Items 4950 out of 8005 code: 200 on Game:582810
Items 5040 out of 8005 code: 200 on Game:582810
Items 5130 out of 8005 code: 200 on Game:582810
Items 5220 out of 8005 code: 200 on Game:582810
Items 5310 out of 8005 code: 200 on Game:582810
Items 5400 out of 8005 code: 200 on Game:582810
Items 5490 out of 8005 code: 200 on Game:582810
Items 5580 out of 8005 code: 200 on Game:582810
Items 5670 out of 8005 code: 200 on Game:582810
Items 5760 out of 8005 code: 200 on Game:582810
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 5850 out of 8005 code: 200 on Game:582810
Items 5940 out of 8005 code: 200 on Game:582810
Items 6030 out of 8005 code: 200 on Game:582810
Items 6120 out of 8005 code: 200 on Game:582810
Items 6210 out of 8005 code: 200 on Game

Items 90 out of 36 code: 200 on Game:615050
len(list(set(allItemNames))):  36 len(allItemNames):  36
Items 0 out of 196 code: 200 on Game:397900
Items 90 out of 196 code: 200 on Game:397900
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 180 out of 196 code: 200 on Game:397900
Items 270 out of 196 code: 200 on Game:397900
len(list(set(allItemNames))):  184 len(allItemNames):  216
Items 0 out of 47 code: 200 on Game:722960
Items 90 out of 47 code: 200 on Game:722960
len(list(set(allItemNames))):  47 len(allItemNames):  47
Items 0 out of 1 code: 200 on Game:797180
Items 90 out of 1 code: 200 on Game:797180
len(list(set(allItemNames))):  1 len(allItemNames):  1
Items 0 out of 175 code: 200 on Game:302670
Items 90 out of 175 code: 200 on Game:302670
Items 180 out of 175 code: 200 on Game:302670
len(list(set(allItemNames))):  172 len(allItemNames):  185
Items 0 out of 4 code: 200 on Game:1850930
Items 90 out of 4 code: 200 on Game:185093

Items 9630 out of 18764 code: 200 on Game:730
Items 9720 out of 18764 code: 200 on Game:730
Items 9810 out of 18764 code: 200 on Game:730
Items 9900 out of 18764 code: 200 on Game:730
Items 9990 out of 18764 code: 200 on Game:730
Items 10080 out of 18764 code: 200 on Game:730
Items 10170 out of 18764 code: 200 on Game:730
Items 10260 out of 18764 code: 200 on Game:730
Items 10350 out of 18764 code: 200 on Game:730
Items 10440 out of 18764 code: 200 on Game:730
Items 10530 out of 18764 code: 200 on Game:730
Items 10620 out of 18764 code: 200 on Game:730
Items 10710 out of 18764 code: 200 on Game:730
Items 10800 out of 18764 code: 200 on Game:730
Items 10890 out of 18764 code: 200 on Game:730
Items 10980 out of 18764 code: 200 on Game:730
Items 11070 out of 18764 code: 200 on Game:730
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 11160 out of 18764 code: 200 on Game:730
Items 11250 out of 18764 code: 200 on Game:730
Items 11340 out 

Items 0 out of 1 code: 200 on Game:1054900
Items 90 out of 1 code: 200 on Game:1054900
len(list(set(allItemNames))):  1 len(allItemNames):  1
Items 0 out of 28 code: 200 on Game:663920
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 90 out of 28 code: 200 on Game:663920
len(list(set(allItemNames))):  28 len(allItemNames):  28
Items 0 out of 9 code: 200 on Game:278970
Items 90 out of 9 code: 200 on Game:278970
len(list(set(allItemNames))):  9 len(allItemNames):  9
Items 0 out of 8 code: 200 on Game:506730
Items 90 out of 8 code: 200 on Game:506730
len(list(set(allItemNames))):  8 len(allItemNames):  8
Items 0 out of 7 code: 200 on Game:401190
Items 90 out of 7 code: 200 on Game:401190
len(list(set(allItemNames))):  7 len(allItemNames):  7
Items 0 out of 7 code: 200 on Game:874400
Items 90 out of 7 code: 200 on Game:874400
len(list(set(allItemNames))):  7 len(allItemNames):  7
Items 0 out of 25 code: 200 on Game:1862210
Items 90 out o

...

Items 8730 out of 40883 code: 200 on Game:440
Items 8820 out of 40883 code: 200 on Game:440
Items 8910 out of 40883 code: 200 on Game:440
Items 9000 out of 40883 code: 200 on Game:440
Items 9090 out of 40883 code: 200 on Game:440
Items 9180 out of 40883 code: 200 on Game:440
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 9270 out of 40883 code: 200 on Game:440
Items 9360 out of 40883 code: 200 on Game:440
Items 9450 out of 40883 code: 200 on Game:440
Items 9540 out of 40883 code: 200 on Game:440
Items 9630 out of 40883 code: 200 on Game:440
Items 9720 out of 40883 code: 200 on Game:440
Items 9810 out of 40883 code: 200 on Game:440
Items 9900 out of 40883 code: 200 on Game:440
Items 9990 out of 40883 code: 200 on Game:440
Items 10080 out of 40883 code: 200 on Game:440
Items 10170 out of 40883 code: 200 on Game:440
Items 10260 out of 40883 code: 200 on Game:440
Items 10350 out of 40883 code: 200 on Game:440
Items 10440 out of 40883 c

Items 23220 out of 40883 code: 200 on Game:440
Items 23310 out of 40883 code: 200 on Game:440
Items 23400 out of 40883 code: 200 on Game:440
Items 23490 out of 40883 code: 200 on Game:440
Items 23580 out of 40883 code: 200 on Game:440
Items 23670 out of 40883 code: 200 on Game:440
Items 23760 out of 40883 code: 200 on Game:440
Items 23850 out of 40883 code: 200 on Game:440
Items 23940 out of 40883 code: 200 on Game:440
Items 24030 out of 40883 code: 200 on Game:440
Items 24120 out of 40883 code: 200 on Game:440
Items 24210 out of 40883 code: 200 on Game:440
Items 24300 out of 40883 code: 200 on Game:440
Items 24390 out of 40883 code: 200 on Game:440
Items 24480 out of 40883 code: 200 on Game:440
Items 24570 out of 40883 code: 200 on Game:440
Items 24660 out of 40883 code: 200 on Game:440
Items 24750 out of 40883 code: 200 on Game:440
Items 24840 out of 40883 code: 200 on Game:440
Items 24930 out of 40883 code: 200 on Game:440
##### Got 429 Error, sleeping for 1 minute #####
##### Decre

Items 37890 out of 40883 code: 200 on Game:440
Items 37980 out of 40883 code: 200 on Game:440
Items 38070 out of 40883 code: 200 on Game:440
Items 38160 out of 40883 code: 200 on Game:440
Items 38250 out of 40883 code: 200 on Game:440
Items 38340 out of 40883 code: 200 on Game:440
Items 38430 out of 40883 code: 200 on Game:440
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 38520 out of 40883 code: 200 on Game:440
Items 38610 out of 40883 code: 200 on Game:440
Items 38700 out of 40883 code: 200 on Game:440
Items 38790 out of 40883 code: 200 on Game:440
Items 38880 out of 40883 code: 200 on Game:440
Items 38970 out of 40883 code: 200 on Game:440
Items 39060 out of 40883 code: 200 on Game:440
Items 39150 out of 40883 code: 200 on Game:440
Items 39240 out of 40883 code: 200 on Game:440
Items 39330 out of 40883 code: 200 on Game:440
Items 39420 out of 40883 code: 200 on Game:440
Items 39510 out of 40883 code: 200 on Game:440
Items 39600

Items 7110 out of 12833 code: 200 on Game:304930
Items 7200 out of 12833 code: 200 on Game:304930
Items 7290 out of 12833 code: 200 on Game:304930
Items 7380 out of 12833 code: 200 on Game:304930
Items 7470 out of 12833 code: 200 on Game:304930
Items 7560 out of 12833 code: 200 on Game:304930
Items 7650 out of 12833 code: 200 on Game:304930
Items 7740 out of 12833 code: 200 on Game:304930
Items 7830 out of 12833 code: 200 on Game:304930
Items 7920 out of 12833 code: 200 on Game:304930
Items 8010 out of 12833 code: 200 on Game:304930
Items 8100 out of 12833 code: 200 on Game:304930
Items 8190 out of 12833 code: 200 on Game:304930
Items 8280 out of 12833 code: 200 on Game:304930
Items 8370 out of 12833 code: 200 on Game:304930
Items 8460 out of 12833 code: 200 on Game:304930
Items 8550 out of 12833 code: 200 on Game:304930
Items 8640 out of 12833 code: 200 on Game:304930
Items 8730 out of 12833 code: 200 on Game:304930
Items 8820 out of 12833 code: 200 on Game:304930
Items 8910 out of 12

Items 90 out of 1 code: 200 on Game:1890340
len(list(set(allItemNames))):  1 len(allItemNames):  1
Items 0 out of 182 code: 200 on Game:1157720
Items 90 out of 182 code: 200 on Game:1157720
Items 180 out of 182 code: 200 on Game:1157720
Items 270 out of 182 code: 200 on Game:1157720
len(list(set(allItemNames))):  182 len(allItemNames):  194
Items 0 out of 1 code: 200 on Game:714910
Items 90 out of 1 code: 200 on Game:714910
len(list(set(allItemNames))):  1 len(allItemNames):  1
Items 0 out of 204 code: 200 on Game:771950
Items 90 out of 204 code: 200 on Game:771950
Items 180 out of 204 code: 200 on Game:771950
Items 270 out of 204 code: 200 on Game:771950
len(list(set(allItemNames))):  168 len(allItemNames):  224
Items 0 out of 4 code: 200 on Game:722670
Items 90 out of 4 code: 200 on Game:722670
len(list(set(allItemNames))):  4 len(allItemNames):  4
##### Got 429 Error, sleeping for 1 minute #####
##### Decreasing requestsBefore429 to $38#####
Items 0 out of 109 code: 200 on Game:1245

In [46]:
for gameID in ids:
    #if you want to end at a specif date pass a string with the date in %m/%d/%y format
    #as the second parameter
    complete_game_processing(gameID)

print('All item data collected')

done with the scrapping moving to pass data to dataframes
proceding to write data in files
Saved 1889410
2  out of  8 code: 400 on Game:777010 item: Card for LoneStar
Card%20for%20LoneStar bad no data 

3  out of  8 code: 400 on Game:777010 item: luckover
luckover bad no data 

4  out of  8 code: 400 on Game:777010 item: M̲̰͗ͨ͐ͪḭ̼̼̜ͨ̏̋̐͢n̪̲̞̲͈͚̟̫̖̓̊̾͐͐͆͆e͚͖̱̝͈͆̓̉ͮ̓ͦ͆͘p̡̣̜̯̹̱͖̩͋ͬ͗o̝͔̞̲͕̠̤͐ͧͪ̽̑ş͖̥̖̻̻͍̼̏̓̑ͫ̏̏͡t̏҉̩̗̱̤̞̯̘o̖͎̠̦̦̹͙͗̚̕l̞̲͙̼̺̗̫̖̞ͩͯ̌͛̉̋̔͛̚
M̲̰͗ͨ͐ͪḭ̼̼̜ͨ̏̋̐͢n̪̲̞̲͈͚̟̫̖̓̊̾͐͐͆͆e͚͖̱̝͈͆̓̉ͮ̓ͦ͆͘p̡̣̜̯̹̱͖̩͋ͬ͗o̝͔̞̲͕̠̤͐ͧͪ̽̑ş͖̥̖̻̻͍̼̏̓̑ͫ̏̏͡t̏҉̩̗̱̤̞̯̘o̖͎̠̦̦̹͙͗̚̕l̞̲͙̼̺̗̫̖̞ͩͯ̌͛̉̋̔͛̚ bad no data 

5  out of  8 code: 400 on Game:777010 item: Rectangulars favorite image
Rectangulars%20favorite%20image bad no data 

6  out of  8 code: 400 on Game:777010 item: hypetrain
hypetrain bad no data 

7  out of  8 code: 400 on Game:777010 item: ★ Советский нож | Дедушкины трущобы (Прямо с завода)
★%20Советский%20нож%20%7C%20Дедушкины%20трущобы%20%28Прямо%20с%20завода%29 bad no data 

8  out of  8 code: 40

58  out of  324 code: 400 on Game:866510 item: Aenele | Signature Pack #1
Aenele%20%7C%20Signature%20Pack%20#1 bad no data 

59  out of  324 code: 400 on Game:866510 item: Flag of United States of America
Flag%20of%20United%20States%20of%20America bad no data 

60  out of  324 code: 400 on Game:866510 item: Stolen Turquoise asteroid Legendary quality
Stolen%20Turquoise%20asteroid%20Legendary%20quality bad no data 

61  out of  324 code: 400 on Game:866510 item: Stolen Royal fuchsia asteroid Rare quality
Stolen%20Royal%20fuchsia%20asteroid%20Rare%20quality bad no data 

62  out of  324 code: 400 on Game:866510 item: Сhocolate macaroon
Сhocolate%20macaroon bad no data 

63  out of  324 code: 400 on Game:866510 item: Stolen Deep purple asteroid Common quality
Stolen%20Deep%20purple%20asteroid%20Common%20quality bad no data 

64  out of  324 code: 400 on Game:866510 item: Forged by ancients masters Royal fuchsia asteroid Common quality
Forged%20by%20ancients%20masters%20Royal%20fuchsia%20a

120  out of  324 code: 400 on Game:866510 item: Hourglass
Hourglass bad no data 

121  out of  324 code: 400 on Game:866510 item: Acquired Fiery red asteroid Legendary quality
Acquired%20Fiery%20red%20asteroid%20Legendary%20quality bad no data 

122  out of  324 code: 400 on Game:866510 item: Acquired Blue-purple asteroid Legendary quality
Acquired%20Blue-purple%20asteroid%20Legendary%20quality bad no data 

123  out of  324 code: 400 on Game:866510 item: Inherited Deep purple asteroid Legendary quality
Inherited%20Deep%20purple%20asteroid%20Legendary%20quality bad no data 

124  out of  324 code: 400 on Game:866510 item: Angry — AniPack#6
Angry%20—%20AniPack#6 bad no data 

125  out of  324 code: 400 on Game:866510 item: xehhex | Signature Pack #1
xehhex%20%7C%20Signature%20Pack%20#1 bad no data 

126  out of  324 code: 400 on Game:866510 item: Anime Avatar #6
Anime%20Avatar%20#6 bad no data 

127  out of  324 code: 400 on Game:866510 item: Acquired Coal asteroid Common quality
Acquir

185  out of  324 code: 400 on Game:866510 item: Smile — AniPack#7
Smile%20—%20AniPack#7 bad no data 

186  out of  324 code: 400 on Game:866510 item: Acquired Sand gray-brown asteroid Legendary quality
Acquired%20Sand%20gray-brown%20asteroid%20Legendary%20quality bad no data 

187  out of  324 code: 400 on Game:866510 item: Acquired Georgian pink asteroid Legendary quality
Acquired%20Georgian%20pink%20asteroid%20Legendary%20quality bad no data 

188  out of  324 code: 400 on Game:866510 item: Forged by ancients masters Turquoise asteroid Rare quality
Forged%20by%20ancients%20masters%20Turquoise%20asteroid%20Rare%20quality bad no data 

189  out of  324 code: 400 on Game:866510 item: Forged by ancients masters Caribbean green asteroid Rare quality
Forged%20by%20ancients%20masters%20Caribbean%20green%20asteroid%20Rare%20quality bad no data 

190  out of  324 code: 400 on Game:866510 item: Acquired Pacific blue asteroid Legendary quality
Acquired%20Pacific%20blue%20asteroid%20Legendary%20

249  out of  324 code: 400 on Game:866510 item: Stolen Coal asteroid Rare quality
Stolen%20Coal%20asteroid%20Rare%20quality bad no data 

250  out of  324 code: 400 on Game:866510 item: Lemon macaroon
Lemon%20macaroon bad no data 

251  out of  324 code: 400 on Game:866510 item: Inherited Georgian pink asteroid Unreachable quality
Inherited%20Georgian%20pink%20asteroid%20Unreachable%20quality bad no data 

252  out of  324 code: 400 on Game:866510 item: Inherited Green Crayola asteroid Legendary quality
Inherited%20Green%20Crayola%20asteroid%20Legendary%20quality bad no data 

253  out of  324 code: 400 on Game:866510 item: Bleh — AniPack#5
Bleh%20—%20AniPack#5 bad no data 

254  out of  324 code: 400 on Game:866510 item: Sleeping — AniPack#6
Sleeping%20—%20AniPack#6 bad no data 

255  out of  324 code: 400 on Game:866510 item: Inherited Blue-purple asteroid Rare quality
Inherited%20Blue-purple%20asteroid%20Rare%20quality bad no data 

256  out of  324 code: 400 on Game:866510 item: Hi

...

663  out of  5760 code: 400 on Game:582810 item: 10 Centesimi | cifBFv
10%20Centesimi%20%7C%20cifBFv bad no data 

664  out of  5760 code: 400 on Game:582810 item: Zloty | c10674
Zloty%20%7C%20c10674 bad no data 

665  out of  5760 code: 400 on Game:582810 item: Pfennig | cdNWNp
Pfennig%20%7C%20cdNWNp bad no data 

666  out of  5760 code: 400 on Game:582810 item: Penny | cIiebb
Penny%20%7C%20cIiebb bad no data 

667  out of  5760 code: 400 on Game:582810 item: 5 Roubles | cegdhV
5%20Roubles%20%7C%20cegdhV bad no data 

668  out of  5760 code: 400 on Game:582810 item: 2 Heller | ckxemC
2%20Heller%20%7C%20ckxemC bad no data 

669  out of  5760 code: 400 on Game:582810 item: 20 Corona | cxJERL
20%20Corona%20%7C%20cxJERL bad no data 

670  out of  5760 code: 400 on Game:582810 item: Thaler | cIiaBh
Thaler%20%7C%20cIiaBh bad no data 

671  out of  5760 code: 400 on Game:582810 item: 8 Florins-20 Francs | cWtLti
8%20Florins-20%20Francs%20%7C%20cWtLti bad no data 

672  out of  5760 code: 400

739  out of  5760 code: 400 on Game:582810 item: Penny | cXArBt
Penny%20%7C%20cXArBt bad no data 

740  out of  5760 code: 400 on Game:582810 item: 3 Kreuzer | cquiUc
3%20Kreuzer%20%7C%20cquiUc bad no data 

741  out of  5760 code: 400 on Game:582810 item: 10 Cash | cWMMtu
10%20Cash%20%7C%20cWMMtu bad no data 

742  out of  5760 code: 400 on Game:582810 item: Ecu d'or | cvAcEl
Ecu%20d%27or%20%7C%20cvAcEl bad no data 

743  out of  5760 code: 400 on Game:582810 item: 10 Groszy | c10559
10%20Groszy%20%7C%20c10559 bad no data 

744  out of  5760 code: 400 on Game:582810 item: Rouble | cByQQS
Rouble%20%7C%20cByQQS bad no data 

745  out of  5760 code: 400 on Game:582810 item: 10 Kopeks | cxBWai
10%20Kopeks%20%7C%20cxBWai bad no data 

746  out of  5760 code: 400 on Game:582810 item: 1-2 Kopek | cdYgJs
1-2%20Kopek%20%7C%20cdYgJs bad no data 

747  out of  5760 code: 400 on Game:582810 item: 10 Centesimi | cmcRNb
10%20Centesimi%20%7C%20cmcRNb bad no data 

748  out of  5760 code: 400 on Game

814  out of  5760 code: 400 on Game:582810 item: Dollar | cGapcc
Dollar%20%7C%20cGapcc bad no data 

815  out of  5760 code: 400 on Game:582810 item: Sesino | cUrvry
Sesino%20%7C%20cUrvry bad no data 

816  out of  5760 code: 400 on Game:582810 item: 30 Sols | cvDFkv
30%20Sols%20%7C%20cvDFkv bad no data 

817  out of  5760 code: 400 on Game:582810 item: 5 Centesimi | cTWyTX
5%20Centesimi%20%7C%20cTWyTX bad no data 

818  out of  5760 code: 400 on Game:582810 item: Louis d'Or | csdgkJ
Louis%20d%27Or%20%7C%20csdgkJ bad no data 

819  out of  5760 code: 400 on Game:582810 item: 5 Francs | cpUWoK
5%20Francs%20%7C%20cpUWoK bad no data 

820  out of  5760 code: 400 on Game:582810 item: ECU - 6 Livres | cIfkHg
ECU%20-%206%20Livres%20%7C%20cIfkHg bad no data 

821  out of  5760 code: 400 on Game:582810 item: 2 Mark | cmrMBg
2%20Mark%20%7C%20cmrMBg bad no data 

822  out of  5760 code: 400 on Game:582810 item: 20 Francs | cADSMI
20%20Francs%20%7C%20cADSMI bad no data 

823  out of  5760 code: 4

889  out of  5760 code: 400 on Game:582810 item: $5 - Half Eagle | clHebu
$5%20-%20Half%20Eagle%20%7C%20clHebu bad no data 

890  out of  5760 code: 400 on Game:582810 item: 5 Groszy | c10573
5%20Groszy%20%7C%20c10573 bad no data 

891  out of  5760 code: 400 on Game:582810 item: 40 Lire | cDhmPI
40%20Lire%20%7C%20cDhmPI bad no data 

892  out of  5760 code: 400 on Game:582810 item: Quarter | cHbheX
Quarter%20%7C%20cHbheX bad no data 

893  out of  5760 code: 400 on Game:582810 item: 20 Kreuzer | ckgyTT
20%20Kreuzer%20%7C%20ckgyTT bad no data 

894  out of  5760 code: 400 on Game:582810 item: 1-2 Ecu | cXxBpY
1-2%20Ecu%20%7C%20cXxBpY bad no data 

895  out of  5760 code: 400 on Game:582810 item: 2 Kopeks | cdhgyi
2%20Kopeks%20%7C%20cdhgyi bad no data 

896  out of  5760 code: 400 on Game:582810 item: 5 Mark | cXGIQu
5%20Mark%20%7C%20cXGIQu bad no data 

897  out of  5760 code: 400 on Game:582810 item: Thaler | cDvBVv
Thaler%20%7C%20cDvBVv bad no data 

898  out of  5760 code: 400 on Ga

KeyboardInterrupt: 

In [ ]:
with open('log.txt','w') as f:
    f.write(log)